<a href="https://colab.research.google.com/github/the-goat-man/Election-Script/blob/main/Election_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## *EDITABLE VARIABLES*


Please make sure to have the `.csv` file has ranking columns named after the candidates they represent. (i.e. if the candidate is *Ritvik Garg*, a senior, the column title will be "*Ritvik Garg*")<br>

The row that includes the candidate names should be right above the choices, but not the first row thanks to the `start_row` variable. Similarly, the ranking columns must be sequential, but do not need to start at the first column thanks to the `start_col` variable.
&nbsp;  
&nbsp;  

To import the `.csv` file, download the file onto your computer, then go to `Files` on the sidebar and upload the file to session storage.<br>
If there are issues when running the program related to the file path, check here to make sure the `.csv` file is present. *If the issue persists, go to the `Toolbar`, select `Runtime`, then choose `Disconnect and delete runtime` and restart the process.*
&nbsp;  
&nbsp;  

Also, confirm that the ranking values are **exactly** as inputted below (case insensitive) in the order of *highest ranked* to *lowest ranked*.
&nbsp;  
&nbsp;  

If you would like the program to automatically include a non-senior,
change `check_seniors` to `True` and set the column titles for the non-seniors to end with a character, specified in the variable called `senior_signifier`.
&nbsp;  
&nbsp;  

You can also color-code the candidates! To do so, set `color_names` to `True`. Whether you want color-coded names or not, all names will be bolded and underlined, and other words will be colored to help readability.<br>

*NOTE: Colors should work, but may vary from browser to browser, so do not worry if colors do not show up.*

---

Once the variables are set to your liking, go to the `Toolbar`, select `Runtime`, then choose `Run All`. Then, go to the `Output Here` section to see the results of the election.
&nbsp;  
&nbsp;  

If any errors are encountered that do not state an issue with the `Editable Variables`, please contact **Ritvik Garg** for assistance.
&nbsp;  
&nbsp;  

Library Dependencies: Pandas, Random, OS, SYS

In [13]:
# the location of the voting.csv file in the colab (keep the quotes, parenthesis, and the 'r' in front of the quotes)
path: str = (r"/content/voting.csv")


# the row and column the first candidate's name is in, in numbers (if the first column is D, enter '4')
start_row: int = 2
start_col: int = 2

# the amount of candidates in the vote
num_candidates: int = 8

# the amount of seats to elect for
num_seats: int = 3

# the text in the table for the choices, in order of (highest ranked to lowest ranked) -- this value, both here and in the .csv input, are case insensitive
ranking_names: list = ["fiRSt cHoicE", "sEcOnd chOIcE", "third choice", "FOURTH CHOICE", "Fifth Choice", "sixth choice", "seventh choice", "eighth choice"]

# whether the program's user wants to filter out seniors, and what end character they are signified by, or let the program do so
check_seniors: bool = True
senior_signifier: str = '*' # does not matter if check_seniors is False

# enable colors for the candidate names in the output - only works when there are 6 or less candidates
color_names: bool = True

## PLEASE <u>*DO NOT*</u> EDIT CODE IN THIS SECTION! <u>**Things might break if you do!**</u>

In [14]:
# class containing ANSI shortcuts for text formatting

class color:
    # PURPLE = '\033[95m'
    # CYAN = '\033[96m'
    # BLUE = '\033[94m'
    # YELLOW = '\033[93m'
    # DARK_CYAN = '\033[36m'
    # DARK_BLUE = '\033[34m'
    COLORS: list = ['\033[95m', '\033[96m', '\033[94m', '\033[93m', '\033[36m', '\033[34m']
    RED: str = '\033[91m'
    GREEN: str = '\033[92m'
    BOLD: str = '\033[1m'
    ITALICS: str = '\033[3m'
    UNDERLINE: str = '\033[4m'
    BLACK_BG: str = '\033[40m'
    END: str = '\033[0m'

In [ ]:
# all imports

import pandas as pd
from random import choice, shuffle, seed
from os.path import exists
from sys import exit

In [ ]:
# refresh seed

seed()

In [15]:
# create functions and logic

ranking_names = [name.lower() for name in ranking_names]

def confirm_inputs() -> None:
    """
    Validates input parameters to ensure they meet the required conditions for the election.
    Exits the program if any of the conditions are not met.
    """
    if len(ranking_names) < num_candidates:
        exit("Something went wrong with the amount of ranking names. There should be as many ranking names as candidates. Please try again.")
    elif num_candidates < num_seats:
        exit("Something went wrong with the amount of seats. There should be fewer seats than candidates. Please try again.")
    elif num_candidates == 1:
        exit("Something went wrong with the amount of candidates. There should be at least two candidates. Please try again.")
    elif num_candidates == num_seats:
        exit("Something went wrong with the amount of candidates. The amount of seats is equal to the amount of candidates, and thus the election does not need to happen.")
    elif not exists(path):
        exit("Something went wrong with the path. Please check that the path exists and try again.")

def fix_value(ranking: str) -> float:
    """
    Converts a ranking string to its corresponding numerical value based on its position in the ranking_names list.
    Returns None if the ranking is NaN.
    """
    if pd.isna(ranking):
        return None
    ranking = ranking.lower()
    for i in range(len(ranking_names)):
        if ranking_names[i] == ranking:
            return i + 1
    return None

def get_quota(num_votes: int) -> float:
    """
    Calculates the quota required to win a seat in the election based on the number of votes and seats.
    """
    return int(float(num_votes) / (num_seats + 1) * 100) / 100.0

def break_tie(tied_candidates: list, candidates: list) -> str:
    """
    Randomly selects a candidate from the list in case of a tie.
    """
    if (len(tied_candidates) == 1):
        return tied_candidates[0]

    print("\tTiebreaker!")
    to_print: str = ""
    for i in range(len(tied_candidates)):
        candidate: str = tied_candidates[i]
        to_print += set_candidate_color(candidate, candidates)
        if len(tied_candidates) == 2 and i == 0:
            to_print += " and "
        elif len(tied_candidates) > 2 and i == len(tied_candidates) - 2:
            to_print += ", and "
        elif len(tied_candidates) > 2 and i != len(tied_candidates) - 1:
            to_print += ", "
    selected_candidate: str = choice(tied_candidates)
    print("\t" + set_candidate_color(selected_candidate, candidates) + " has been selected between " + to_print + ".\n")
    return selected_candidate

def set_colors(candidates: list) -> list:
    """
    Assigns colors to the candidates for display purposes.
    """
    shuffle(color.COLORS)
    for i in range(len(candidates)):
        if color_names and len(candidates) <= len(color.COLORS):
            candidates[i] = color.ITALICS + color.UNDERLINE + color.COLORS[i] + candidates[i] + color.END
        else:
            candidates[i] = color.ITALICS + color.UNDERLINE + candidates[i] + color.END
    return candidates

def allocate_first_votes(df: pd.DataFrame) -> dict:
    """
    Allocates the first preference votes to each candidate.
    """
    votes: dict = {}
    for index, vote in df.iterrows():
        candidate = vote.idxmin() if not vote.isna().all() else None
        if candidate is not None:
            if candidate not in votes:
                votes[candidate] = []
            votes[candidate].append((vote, 1))
    return votes

def set_candidate_color(candidate: str, candidates: list) -> str:
    """
    Returns the candidate's name with its assigned color.
    """
    for i in range(len(candidates)):
        if color_names and len(candidates) <= len(color.COLORS):
            if candidates[i][13:-4] == candidate:
                return '\033[40m' + candidates[i]
        else:
            if candidates[i][8:-4] == candidate:
                return candidates[i]
    return None

def set_scores(vote_assign: dict) -> dict:
    """
    Calculates the scores for each candidate based on the votes assigned to them.
    """
    scores: dict = {}
    for candidate in vote_assign:
        scores[candidate] = 0.0
        for vote, vote_val in vote_assign[candidate]:
            scores[candidate] += vote_val
    return scores

def print_round_results(victor: str, candidates: list, candidate_scores: dict, quota: int, round_num: int) -> None:
    """
    Prints the results of each round of voting.
    """
    print(color.BOLD + "Round " + str(round_num) + ":" + color.END)
    for candidate in candidate_scores:
        candidate_score = candidate_scores[candidate]
        if candidate == victor:
            candidate = set_candidate_color(candidate, candidates)
            print("\t" + candidate + " has " + color.BOLD + color.GREEN + "won" + color.END + " with " + str(candidate_score) + " points!")
        elif candidate_score >= quota:
            candidate = set_candidate_color(candidate, candidates)
            print("\t" + candidate + " has " + color.BOLD + color.RED + "not yet won" + color.END + " with " + str(candidate_score) + " points.")
        else:
            candidate = set_candidate_color(candidate, candidates)
            print("\t" + candidate + " has " + color.BOLD + color.RED + "not won" + color.END + " with " + str(candidate_score) + " points.")
    print("\n")

def add_to_victors(quota: int, candidate_scores: dict, num_current_victors: int, num_seats: int, candidates: list) -> str | None:
    """
    Returns the candidate who meets the quota with the highest score, or None if none have.
    """
    max_score: int = 0
    best_candidates: list = list()
    for candidate in candidate_scores:
        if candidate_scores[candidate] >= quota:
            if candidate_scores[candidate] > max_score:
                max_score = candidate_scores[candidate]
                best_candidates = [candidate]
            elif candidate_scores[candidate] == max_score:
                best_candidates.append(candidate)
    if max_score == 0:
        return None
    return break_tie(best_candidates, candidates)

def add_victors(current_victors: list, candidates: list, candidate_scores: dict, quota: int, round_num: int) -> str | None:
    """
    Adds new victor to the current list of victors and prints the round results.
    """
    new_victor: str = add_to_victors(quota, candidate_scores, len(current_victors), num_seats, candidates)
    print_round_results(new_victor, candidates, candidate_scores, quota, round_num)
    if new_victor != None:
        print("\t" + set_candidate_color(new_victor, candidates) + " has been " + color.GREEN + "added to the winners" + color.END + ".\n")
    return new_victor

def remove_worst_candidate(candidate_scores: list, candidates: list) -> str:
    """
    Removes the worst-performing candidate from the list of candidates.
    """
    list_to_remove = []
    min_score = candidate_scores[list(candidate_scores.keys())[0]]
    for candidate in candidate_scores:
        if candidate_scores[candidate] < min_score:
            min_score = candidate_scores[candidate]
            list_to_remove = [candidate]
        elif candidate_scores[candidate] == min_score:
            list_to_remove.append(candidate)
    candidate_to_remove: str = break_tie(list_to_remove, candidates)
    print("\t" + set_candidate_color(candidate_to_remove, candidates) + " has been " + color.RED + "removed for lowest score" + color.END + ".\n")
    return candidate_to_remove

def get_surplus(quota: int, candidate_score: int) -> float:
    """
    Calculates the surplus votes for a candidate who exceeds the quota.
    """
    return float(candidate_score - quota) / candidate_score

def reassign_votes(candidate: str, victors: list, eliminated: list, vote_assign: dict, quota: int, candidate_scores: dict, no_new_victors: bool) -> dict:
    """
    Reassigns the surplus votes of new victors to other candidates.
    """
    surplus_fraction: float = 0.0
    if no_new_victors:
        surplus_fraction = 1.0
    else:
        surplus_fraction = get_surplus(quota, candidate_scores[candidate])
    if surplus_fraction == 0.0:
        del vote_assign[candidate]
        return vote_assign
    for vote, vote_val in vote_assign[candidate]:
        vote_val *= surplus_fraction

        new_rank = vote[candidate] + 1
        vote = vote.drop(labels=candidate)
        if new_rank not in vote.values:
            continue
        new_candidate = vote[vote == new_rank].index[0]
        while new_candidate in victors or new_candidate in eliminated:
            new_rank += 1
            vote = vote.drop(labels=new_candidate)
            if new_rank not in vote.values:
                break
            new_candidate = vote[vote == new_rank].index[0]
        if new_rank not in vote.values:
            continue
        if new_candidate not in vote_assign:
            vote_assign[new_candidate] = []
        vote_assign[new_candidate].append((vote, vote_val))
    del vote_assign[candidate]
    return vote_assign

def remove_last_senior(victors: list, eliminated: list, candidates: list, candidate_scores: dict) -> list:
    """
    Replaces the last senior victor if there are too many seniors elected.
    """
    to_replace = victors[-1]
    print("Due to there being too many seniors elected, " + set_candidate_color(to_replace, candidates).replace("*", "") + " is " + color.RED + "being replaced" + color.END + ".\n")
    runner_up = ""
    score = 0
    for candidate in candidate_scores:
        if candidate[-1] == senior_signifier and candidate_scores[candidate] > score:
            score = candidate_scores[candidate]
            runner_up = candidate
    if score == 0 and len(eliminated) == 0:
        print("There are no non-seniors in the pool. Thus, there will be no change to the final candidates.")
    elif score != 0:
        print(set_candidate_color(runner_up, candidates) + " is " + color.GREEN + "replacing them" + color.END + ".")
        del victors[-1]
        victors.append(runner_up)
    else:
        for candidate in reversed(eliminated):
            if candidate[-1] == senior_signifier:
                print(set_candidate_color(candidate, candidates) + " is " + color.GREEN + "replacing them" + color.END + ".")
                del victors[-1]
                victors.append(candidate)
                eliminated.remove(candidate)
                break
    return victors

def print_winners(victors: list, candidates: list) -> None:
    """
    Prints the final list of elected candidates.
    """
    to_print = ""
    if len(victors) == 1:
        to_print += "\n\nThe winner of the election has been decided!\n"
    else:
        to_print += "\n\nThe winners of the election have been decided!\n"
    for i in range(len(victors)):
        victor = victors[i]
        to_print += set_candidate_color(victor, candidates).replace("*", "")
        if len(victors) == 2 and i == 0:
            to_print += " and "
        elif len(victors) > 2 and i == len(victors) - 2:
            to_print += ", and "
        elif len(victors) > 2 and i != len(victors) - 1:
            to_print += ", "

    if len(victors) == 1:
        to_print += " has been elected! Congratulations!"
    else:
        to_print += " have been elected! Congratulations!"

    print(to_print)

def check_if_over(victors: list, eliminated: list, num_seats: int, candidates: list, candidate_scores: dict) -> bool:
    """
    Checks if the election process is over by verifying the number of victors and seniority conditions.
    """
    if len(victors) > num_seats:
        exit("Something went wrong in the program. Contact Ritvik Garg with the following error message: 'More victors than seats.'")
    elif len(victors) < num_seats:
        return False
    if not check_seniors:
        return True
    for victor in victors:
        if victor[-1] == senior_signifier:
            return True
    victors = remove_last_senior(victors, eliminated, candidates, candidate_scores)
    return True

def filter_votes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Removes votes that have multiple candidates with the same ranking, unless that ranking is the lowest.
    """
    removed_votes: int = 0
    for index, vote in df.iterrows():
        to_eliminate: bool = False
        on_block: bool = False
        final_i: int = 0

        for i in range(1, num_candidates + 1):
            i_count: int = 0
            for ranking in vote:
                if ranking == i:
                    final_i = i
                    if on_block:
                        to_eliminate = True
                        break
                    else:
                        i_count += 1
            if i_count > 1:
                on_block = True
            if to_eliminate:
                break

        if to_eliminate:
            df = df.drop(index)
            removed_votes += 1
        elif not to_eliminate and on_block:
            for i in range(num_candidates):
                if vote.iloc[i] == final_i:
                    df.at[index, df.columns.tolist()[i]] = None

    if removed_votes > 0:
        print(str(removed_votes) + " votes removed due to multiple candidates under the same choice.\n")
    df = df.reset_index()
    df.index = df.index + 1
    df = df.drop(columns=['index'])
    return df

def set_df() -> pd.DataFrame:
    """
    Loads the election data from a CSV file and prepares the DataFrame for processing.
    """
    global start_row
    global start_col
    start_row -= 1
    start_col -= 1
    confirm_inputs()
    df = pd.read_csv(path, skiprows=start_row)
    df = df.iloc[:, start_col:(num_candidates + start_col)]
    for col in df:
        df[col] = df[col].apply(fix_value)

    return df

def run_election() -> None:
    """
    Main function to run the election process.
    """
    df: pd.DataFrame = set_df()
    # return df
    num_votes: int = df.shape[0]
    df = filter_votes(df) # filter votes after getting count
    candidates: list = set_colors(list(df.columns))
    quota: int = get_quota(num_votes)
    vote_assign: dict = allocate_first_votes(df)
    victors: list = []
    eliminated: list = []
    round_num: int = 0
    while True:
        round_num += 1
        candidate_scores = set_scores(vote_assign)
        new_victor = add_victors(victors, candidates, candidate_scores, quota, round_num)
        if new_victor == None:
            removed_candidate = remove_worst_candidate(candidate_scores, candidates)
            eliminated.append(removed_candidate)
            vote_assign = reassign_votes(removed_candidate, victors, eliminated, vote_assign, quota, candidate_scores, True)
        else:
            victors.append(new_victor)
            if check_if_over(victors, eliminated, num_seats, candidates, candidate_scores):
                print_winners(victors, candidates)
                return
            vote_assign = reassign_votes(new_victor, victors, eliminated, vote_assign, quota, candidate_scores, False)

## *OUTPUT HERE*

In [ ]:
## This is it! ##

run_election()